In [1]:
!pip install neo4j


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


### Import json into neo4j

#### Read json files

In [2]:
import json
import os

db_directory = 'Data/DB'

report_directory = 'Data/Report'

model_directory = 'Data/Model'

# database
db_files = ['Customerservice-DB.json', 'Executive_Management-DB.json', 'Finance-DB.json',
             'HR-DB.json', 'IT-DB.json', 'Marketing-DB.json',
             'Operations-DB.json', 'Product-DB.json', 'R&D-DB.json', 'Sales-DB.json']
report_files = ['customer-satisfaction-survey-analysis-report-metadata-schema.json',
                'employee-productivity-report-metadata-schema.json', 'financial-health-dashboard-report-metadata-schema.json',
                'inventory-management-report-metadata-schema.json', 'sales-performance-dashboard-report-metadata-schema.json']

model_files = ['sales-performance-model-version1.json', 'sales-performance-model-version2.json', 'sales-performance-model-version3.json',
               'inventory-management-model-version1.json', 'inventory-management-model-version2.json', 'inventory-management-model-version3.json',
               'financial-health-model-version1.json', 'financial-health-model-version2.json', 'financial-health-model-version3.json',
               'employee-productivity-model-version1.json', 'employee-productivity-model-version2.json', 'employee-productivity-model-version3.json',
               'customer-satisfaction-model-version1.json', 'customer-satisfaction-model-version2.json', 'customer-satisfaction-model-version3.json']

db_data = {}
report_data = {}
model_data = {}

# save database files
for db in db_files:
    path = os.path.join(db_directory, db)
    with open(path, 'r') as file:
        data = json.load(file)
        db_data[db] = data


# save report files
for report in report_files:
    path = os.path.join(report_directory, report)
    with open(path, 'r') as file:
        data = json.load(file)
        report_data[report] = data

# # # save model files
for model in model_files:
    path = os.path.join(model_directory, model)
    with open(path, 'r') as file:
        data = json.load(file)
        model_data[model] = data

In [3]:
### save the model data element from report json file
model_DE = {}
for report in report_files:
    report_json = report_data[report]
    for section in report_json['sections']:
        for field in section['fields']:
            if field['source'] == 'model':
                name = "DE_" + field["id"]
                model_DE[name] = field['generatedFrom']

#### Database function

In [4]:
def create_database(session, data):
    # Create Database Node
    session.run("MERGE (db:Database {name: $database_name, type: $type})",
                database_name=data['database_name'], type=data['type'])

    # Create Table and Column Nodes and Relationships
    for table in data['tables']:
        if "primary_key" in table:
            session.run("""
                MATCH (db:Database {name: $database_name})
                MERGE (t:Table {name: $table_name})
                SET t.primary_key = $primary_key
                MERGE (db)-[:CONTAINS]->(t)
            """, database_name=data['database_name'], table_name=table['name'], primary_key=table["primary_key"])

        else:
            session.run("""
                MATCH (db:Database {name: $database_name})
                MERGE (t:Table {name: $table_name})
                MERGE (db)-[:CONTAINS]->(t)
            """, database_name=data['database_name'], table_name=table['name'])

        for column in table['columns']:
            session.run("""
                MATCH (t:Table {name: $table_name})
                MERGE (c:Column {name: $column_name, type: $column_type})
                MERGE (t)-[:HAS_COLUMN]->(c)
            """, table_name=table['name'], column_name=column['name'], column_type=column['type'])
        # add primary key edge
        if "primary_key" in table:
            session.run("""
                MATCH (t:Table {name: $table_name})
                MATCH (c:Column {name: $column_name})
                MERGE (t)-[:HAS_PRIMARY_KEY]->(c)
            """, table_name=table['name'], column_name=table["primary_key"])

    # Add Business Group Node
    session.run("MERGE (bg:BusinessGroup {name: $business_group})",
                business_group=data['business_group'])

    # Create Relationships between Business Group and Database
    session.run("""
        MATCH (db:Database {name: $database_name})
        MATCH (bg:BusinessGroup {name: $business_group})
        MERGE (db)-[:ASSOCIATED_WITH]->(bg)
    """, database_name=data['database_name'], business_group=data['business_group'])

    # Add Key Contact Information Nodes and Relationships
    for contact in data['key_contact_information']:
        session.run("""
            MERGE (contact:Contact {name: $name, email: $email})
            WITH contact
            MATCH (bg:BusinessGroup {name: $business_group})
            MERGE (contact)-[:CONTACT_OF]->(bg)
        """, name=contact['name'], email=contact['email'], business_group=data['business_group'])

    # Add Entitlements Nodes and Relationships
    for entitlement in data['entitlements']:
        user_name = entitlement['user_name']
        user_account = entitlement['user_account']
        session.run("""
            MERGE (user:User {name: $name, account: $account})
            SET user.entitlement = $ent
            WITH user
            MATCH (db:Database {name: $database_name})
            MERGE (user)-[:ENTITLED_ON]->(db)
        """, name=user_name, account=user_account, database_name=data['database_name'], ent=entitlement['entitlements'])


#### Reports function

In [5]:
def create_report(session, data):
    print(data['name'])
    # Match or Create Report node
    report_query = """
        MERGE (r:Report {name: $name})
    """
    session.run(report_query, name=data["name"])

    for section in data["sections"]:
        # Match or Create ReportSection node and relationship
        section_query = """
            MATCH (r:Report {name: $report_name})
            MERGE (s:ReportSection {name: $section_name})
            MERGE (s)-[:PART_OF]->(r)
        """
        session.run(section_query, report_name=data["name"], section_name=section["name"])

        for field in section["fields"]:
            field_query = """
                MATCH (s:ReportSection {name: $section_name})
                MERGE (f:ReportField {name: $field_name, id: $field_id})
                MERGE (f)-[:BELONGS_TO]->(s)
            """
            session.run(field_query, section_name=section["name"], field_id=field["id"], field_name=field["name"])
            # create data element -> report field
            de_query = """
                MATCH (f:ReportField {id: $field_id})
                MERGE (de:DataElement {name: $data_element_name})
                MERGE (de)-[:FEEDS]->(f)
            """
            session.run(de_query, field_id=field["id"], data_element_name="DE_" + field["id"])

            # if the source is column -> create link between data_element and sourcedata(columns)
            if field["source"] == "columns" or field["source"] == "calculation":
                for col in field["sourcedata"]:
                    col_query = """
                        MATCH (c: Column {name: $colname})
                        MATCH (de: DataElement {name: $de_name})
                        SET de.source = $source
                        SET de.generatedFrom = $generatedFrom
                        MERGE (c)-[:TRANSFORMS]->(de)
                    """
                    session.run(col_query, colname = col, de_name="DE_" + field["id"], source=field["source"], generatedFrom=field["generatedFrom"])

            # if the source is model -> do nothing since we already create the link between DE and report
            if field["source"] == "model":

                de_query = """
                    MERGE (de: DataElement {name: $de_name})
                    SET de.source = $source
                    SET de.generatedFrom = $generatedFrom
                """
                session.run(de_query, de_name=field['source_id'], source=field["source"], generatedFrom=field["generatedFrom"])


    # Add users
    for user in data["users"]:
        if user["relationship"] == "Entitled_On":
            user_query = """
                MERGE (u:User {name: $user_name, account: $user_account, entitlement: $ent, role: $role})
                MERGE (r:Report {name: $report_name})
                MERGE (u)-[:ENTITLED_ON]->(r)
            """
            session.run(user_query, user_name=user["name"], user_account=user["user_account"], report_name=data["name"], ent=user["entitlement"], role=user["role"])
        elif user["relationship"] == "Owner":
            user_query = """
                MERGE (u:User {name: $user_name, account: $user_account, entitlement: $ent, role: $role})
                MERGE (r:Report {name: $report_name})
                MERGE (u)-[:OWNS]->(r)
            """
            session.run(user_query, user_name=user["name"], user_account=user["user_account"], report_name=data["name"], ent=user["entitlement"], role=user["role"])
        elif user["relationship"] == "Maintain":
            user_query = """
                MERGE (u:User {name: $user_name, account: $user_account, entitlement: $ent})
                MERGE (r:Report {name: $report_name})
                MERGE (u)-[:MAINTAINS]->(r)
            """
            session.run(user_query, user_name=user["name"], user_account=user["user_account"], report_name=data["name"], ent=user["entitlement"], role=user["role"])

    # Handling business group
    business_group_query = """
        MERGE (bg:BusinessGroup {name: $business_group})
        MERGE (r:Report {name: $report_name})
        MERGE (r)-[:ASSOCIATED_WITH]->(bg)
    """
    session.run(business_group_query, business_group=data["business_group"], report_name=data["name"])


#### Model version function

In [6]:
def create_model_version(session, data):

    # Match or Create ModelVersion node
    metadata_str = json.dumps(data["properties"]["metadata"])
    parameters_str = json.dumps(data["properties"]["model_parameters"])
    top_features_str = json.dumps(data["properties"]["top_features"])
    performance_metrics_str = json.dumps(data["properties"]["performance_metrics"])

    model_version_query = """
        MERGE (mv:ModelVersion {name: $name})
        SET mv.version = $version,
            mv.latest_version = $latest_version,
            mv.metadata = $metadata,
            mv.model_parameters = $model_parameters,
            mv.top_features = $top_features,
            mv.performance_metrics = $performance_metrics,
            mv.model_id = $model_id
    """

    session.run(model_version_query, name=data["name"], version=data["properties"]["version"], latest_version=data["properties"]["latest_version"],
                metadata=metadata_str,
                model_parameters=parameters_str,
                top_features=top_features_str,
                performance_metrics=performance_metrics_str,
                model_id=data["properties"]["model_id"])

    # Match or Create Model node
    model_metadata_str = json.dumps(data["properties"]["version_of"]["model_metadata"])
    model_query = """
        MERGE (m:Model {name: $model_name})
        SET m.model_metadata = $model_metadata,
            m.move_id = $model_id
    """
    session.run(model_query, model_name=data["properties"]["version_of"]["name"],
                model_metadata=model_metadata_str,
                model_id=data["properties"]["version_of"]["model_id"])



    # if it's latest version, create relationship
    if data['properties']['latest_version'] == "True":
        latest_query = """
            MATCH (mv:ModelVersion {name: $name})
            MATCH (m:Model {name: $model_name})
            MERGE (m)-[:VERSION_OF]->(mv)
            MERGE (m)-[:LATEST_VERSION]->(mv)
        """
        session.run(latest_query, name=data["name"], model_name=data["properties"]["version_of"]["name"])
    else:
        # Create relationships between ModelVersion and Model
        relationship_query = """
            MATCH (mv:ModelVersion {name: $name})
            MATCH (m:Model {name: $model_name})
            MERGE (m)-[:VERSION_OF]->(mv)
        """
        session.run(relationship_query, name=data["name"], model_name=data["properties"]["version_of"]["name"])

    # Create relationships between ModelVersion and Input Data Elements
    for element in data["properties"]["input_data_elements"]:
        element_query = """
            MATCH (mv:ModelVersion {name: $name})
            MERGE (de:DataElement {name: $element_name})
            SET de.source = $source
            SET de.generatedFrom = $generatedFrom
            MERGE (de)-[:INPUT_TO]->(mv)
        """
        session.run(element_query, name=data["name"], element_name="DE_" + element["name"], source="columns", generatedFrom=field["generatedFrom"])

        # create relationship of columns and Data element
        for col in element["columns"]:
            col_query = """
                MATCH (de:DataElement {name: $element_name})
                MATCH (col:Column {name: $column_name})
                MERGE (col)-[:TRANSFORMS]->(de)
            """
            session.run(col_query, element_name="DE_" + element["name"], column_name=col)
    # Create relationship between ModelVersion and Output Data Element
    for ele in data["properties"]["output_data_element"]:
        output_element_query = """
            MATCH (mv:ModelVersion {name: $name})
            MERGE (de:DataElement {name: $output_element_name})
            SET de.source = $source
            SET de.generatedFrom = $generatedFrom
            MERGE (mv)-[:PRODUCES]->(de)
        """
        session.run(output_element_query, name=data["name"], output_element_name=ele["id"], source="model", generatedFrom=model_DE[ele['id']])

    # Create user node and relationship

    for user in data["properties"]["users"]:

        if user["relationship"] == "Entitled_On":

            user_query = """
                MATCH (mv:ModelVersion {name: $name})
                MERGE (u:User {name: $user_name, account: $user_account, entitlement: $ent, role: $role})
                MERGE (u)-[:ENTITLED_ON]->(mv)
            """
            session.run(user_query, user_name=user["name"], user_account=user["user_account"], name=data["name"], ent=user["entitlement"], role=user["role"])
        elif user["relationship"] == "Owner":
            user_query = """
                MATCH (mv:ModelVersion {name: $name})
                MERGE (u:User {name: $user_name, account: $user_account, entitlement: $ent, role: $role})
                MERGE (u)-[:OWNS]->(mv)
            """
            session.run(user_query, user_name=user["name"], user_account=user["user_account"], name=data["name"], ent=user["entitlement"], role=user["role"])
        elif user["relationship"] == "Maintain":
            user_query = """
                MATCH (mv:ModelVersion {name: $name})
                MERGE (u:User {name: $user_name, account: $user_account, entitlement: $ent, role: $role})
                MERGE (u)-[:MAINTAINS]->(mv)
            """
            session.run(user_query, user_name=user["name"], user_account=user["user_account"], name=data["name"], ent=user["entitlement"], role=user["role"])



#### run session

In [7]:
from neo4j import GraphDatabase
import os

#RUN FOR WINDOWS
# !pip install python-dotenv
from dotenv import load_dotenv
load_dotenv()

uri = os.getenv('NEO4J_URI')
username = os.getenv('NEO4J_USER')
password = os.getenv('NEO4J_PASSWORD')

driver = GraphDatabase.driver(uri, auth=(username, password))

with driver.session() as session:
  for db in db_files:
    create_database(session, db_data[db])
  for report in report_files:
    create_report(session, report_data[report])
  for model in model_files:
    create_model_version(session, model_data[model])

Customer Satisfaction Survey Analysis
Employee Productivity Report
Financial Health Dashboard
Inventory Management Report
Sales Performance Dashboard


In [8]:
# !pip install langchain_community
# !pip install langchain_openai 
from langchain_community.vectorstores import Neo4jVector
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv

load_dotenv()

BusinessGroup = {
  "index_name": "BusinessGroup_idx",
  "node_label": "BusinessGroup",
  "text_node_properties": ["name"],
  "embedding_node_property": "BusinessGroup_embedding"
}

Column = {
  "index_name": "Column_idx",
  "node_label": "Column",
  "text_node_properties": ["name", "type"],
  "embedding_node_property": "Column_embedding"
}

Contact = {
  "index_name": "Contact_idx",
  "node_label": "Contact",
  "text_node_properties": ["name", "type"],
  "embedding_node_property": "Contact_embedding"
}

Database = {
  "index_name": "Database_idx",
  "node_label": "Database",
  "text_node_properties": ["name", "type"],
  "embedding_node_property": "Database_embedding"
}

DataElement = {
  "index_name": "DataElement_idx",
  "node_label": "DataElement",
  "text_node_properties": ["name", "source", "generatedForm"],
  "embedding_node_property": "DataElement_embedding"
}

Model = {
  "index_name": "Model_idx",
  "node_label": "Model",
  "text_node_properties": ["move_id", "name", "model_metadata"],
  "embedding_node_property": "DataElement_embedding"
}

ModelVersion = {
  "index_name": "ModelVersion_idx",
  "node_label": "ModelVersion",
  "text_node_properties": ["metadata", "latest_version", "performance_metrics", "name", 
                           "model_parameters", "top_features", "model_id", "version"],
  "embedding_node_property": "DataElement_embedding"
}

Report = {
  "index_name": "Report_idx",
  "node_label": "Report",
  "text_node_properties": ["name"],
  "embedding_node_property": "Report_embedding"
}

ReportField = {
  "index_name": "ReportField_idx",
  "node_label": "ReportField",
  "text_node_properties": ["name", "id"],
  "embedding_node_property": "ReportField_embedding"
}

ReportSection = {
  "index_name": "ReportSection_idx",
  "node_label": "ReportSection",
  "text_node_properties": ["name"],
  "embedding_node_property": "ReportSection_embedding"
}

Table = {
  "index_name": "Table_idx",
  "node_label": "Table",
  "text_node_properties": ["name"],
  "embedding_node_property": "Table_embedding"
}

User = {
  "index_name": "User_idx",
  "node_label": "User",
  "text_node_properties": ["name", "account"], #omitted entitlement for now
  "embedding_node_property": "User_embedding"
}

parent = [BusinessGroup, Column, Contact, Database, 
          DataElement, Model, ModelVersion, Report, 
          ReportField, ReportSection, Table, User]

In [9]:
graphs = {}

for i in range(len(parent)):
    # Create the vectorstore for our existing graph
    val = parent[i]["node_label"]
    graphs[f"{val}_embedding_graph"] = Neo4jVector.from_existing_graph(
        embedding=OpenAIEmbeddings(),
        url="neo4j+s://f7a01f50.databases.neo4j.io",
        username="neo4j",
        password="oy-GIeN1YKdVfGKscdImKuQORH_UiGLSRKjt0CJaIs8",
        index_name=parent[i]["index_name"],
        node_label=parent[i]["node_label"],
        text_node_properties=parent[i]["text_node_properties"],
        embedding_node_property=parent[i]["embedding_node_property"],
    )
    print(val)

BusinessGroup
Column
Contact
Database
DataElement
Model
ModelVersion
Report
ReportField
ReportSection
Table
User


In [10]:
#Testing
response = graphs["Column_embedding_graph"].similarity_search(
    "What report fields are downstream of PerformanceScore column?"
)
print(response)

[Document(metadata={}, page_content='\nname: PerformanceScore\ntype: integer'), Document(metadata={}, page_content='\nname: PerformanceReviewDate\ntype: date'), Document(metadata={}, page_content='\nname: PerformanceID\ntype: integer'), Document(metadata={}, page_content='\nname: PerformanceComments\ntype: string')]


In [11]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
vector_qa = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(),
    chain_type="stuff",
    retriever=graphs["Column_embedding_graph"].as_retriever()
)
vector_qa.run(
    "What report fields are downstream of PerformanceScore column?"
)

C:\Users\aksha\AppData\Local\Temp\ipykernel_8928\3580503320.py:4: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm=ChatOpenAI(),
C:\Users\aksha\AppData\Local\Temp\ipykernel_8928\3580503320.py:8: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  vector_qa.run(


'The report fields downstream of the PerformanceScore column are likely PerformanceReviewDate, PerformanceID, and PerformanceComments.'

In [12]:
from langchain.chains import GraphCypherQAChain
from langchain.vectorstores.neo4j_vector import Neo4jVector
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.graphs import Neo4jGraph
graph = Neo4jGraph(
    url=uri,
    username=username,
    password=password
)
graph.refresh_schema()
cypher_chain = GraphCypherQAChain.from_llm(
    cypher_llm = ChatOpenAI(temperature=0, model_name="gpt-4"),
    qa_llm = ChatOpenAI(temperature=0), graph=graph, verbose=True,
    allow_dangerous_requests=True
)
cypher_chain.run(
    "Which users have access to the IT_Database?"
)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (u:User)-[:ENTITLED_ON]->(d:Database) WHERE d.name = 'IT_Database' RETURN u.name
Full Context:
[{'u.name': 'it_manager'}, {'u.name': 'support_specialist'}]

> Finished chain.


'it_manager, support_specialist'